In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
#for defining convolutional layer, pooling layers, fully connected layer,etc
import torch.nn as nn 
#for implementing optimizing algorithm like adam
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
#for image preprocessing and agumentation
from torchvision.transforms import v2 

In [3]:
import cv2
from tqdm import tqdm

In [4]:
from PIL import Image

In [5]:
#fft2 perform 2d fast fourier transform on a 2d array
#fft2 transfrom image from spatial domain to frequency domain
#ifft2 does the inverse
from scipy.fft import fft2, ifft2

In [6]:
import os

In [7]:
#building a custom dataset
class pneumoniaDataset(Dataset):
    def __init__(self,image_path,device='cpu',transform=None):
        self.image_path = image_path
        self.device = device
        self.transform=transform
        self.data = []
        self.load_data()
        
    def load_data(self):
        for label, folder_name in enumerate(os.listdir(self.image_path)):
            folder_path = os.path.join(self.image_path, folder_name)
            if os.path.isdir(folder_path):  # Check if it's a directory
                print(f"Processing folder: {folder_path},{folder_name} as label {label}")
                for image_name in os.listdir(folder_path):
                    if image_name.endswith(('.png', '.jpg', '.jpeg')):  # Ensure valid image files
                        image_path = os.path.join(folder_path, image_name)
                        #print(f"Adding image: {image_path}")
                        self.data.append((image_path, label))
    
    def __len__(self):
            return len(self.data)
            
    def __getitem__(self, index):
        image_path, label = self.data[index]
        image = Image.open(image_path).convert('RGB')  # Load image and convert to RGB
        if self.transform:
            image = self.transform(image)  # Apply transformations
        label = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return image, label

In [8]:
transform = v2.Compose([
    v2.Resize((227, 227)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [9]:
#since i was struglling with automatically appearing of .DS_store in my folder
#which lead to unwanted classifying as class and  labeling it as 0 
# Remove .DS_Store files from the folder and subfolders
for root, dirs, files in os.walk('data'):
    if '.DS_Store' in files:
        os.remove(os.path.join(root, '.DS_Store'))

In [10]:
# Initialize the dataset
train_path=os.path.join('data/train_set')
validation_path=os.path.join('data/val_set')
test_path=os.path.join('data/test_set')
train_dataset = pneumoniaDataset(image_path=train_path, transform=transform)
validation_dataset = pneumoniaDataset(image_path=validation_path, transform=transform)
test_dataset = pneumoniaDataset(image_path=test_path, transform=transform)



Processing folder: data/train_set/bacterial,bacterial as label 0
Processing folder: data/train_set/normal,normal as label 1
Processing folder: data/train_set/viral,viral as label 2
Processing folder: data/val_set/bacterial,bacterial as label 0
Processing folder: data/val_set/normal,normal as label 1
Processing folder: data/val_set/viral,viral as label 2
Processing folder: data/test_set/bacterial,bacterial as label 0
Processing folder: data/test_set/normal,normal as label 1
Processing folder: data/test_set/viral,viral as label 2


In [21]:
len(train_dataset)

7134

In [15]:
# print(len(validation_dataset))
# image, label = train_dataset[1] 
# print(image.shape)         
# print(label)  

793
torch.Size([3, 227, 227])
tensor(0)


In [12]:
#dataloaders
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
validation_dataloader=DataLoader(validation_dataset,batch_size=32,shuffle=False)
test_dataloader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [13]:
#Checking the first batch from the train_loader
for images, labels in train_dataloader:
    print(images.shape)
    print(labels.shape)  
    break 

torch.Size([32, 3, 227, 227])
torch.Size([32])


In [14]:
# Define the model
class AlexNet(nn.Module):
    def __init__(self,input_channels,number_of_classes):
        super(AlexNet, self).__init__() 
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 96, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 6 * 6, 4069)
        self.fc2 = nn.Linear(4069, 4069)
        self.fc3 = nn.Linear(4069, number_of_classes)

        # Other
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(size=5, k=2)
        self.droput = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.maxpool(self.norm(self.relu(self.conv1(x))))  # (B, 96, 27, 27)
        x = self.maxpool(self.norm(self.relu(self.conv2(x))))  # (B, 256, 13, 13)
        x = self.relu(self.conv3(x))                           # (B, 384, 13, 13)
        x = self.relu(self.conv4(x))                           # (B, 384, 13, 13)
        x = self.maxpool(self.relu(self.conv5(x)))             # (B, 256, 6, 6)
        x = self.flatten(x)                                    # (B, 9216)
        x = self.droput(self.relu(self.fc1(x)))                # (B, 4096)
        x = self.droput(self.relu(self.fc2(x)))                # (B, 4096)
        x = self.fc3(x)                                        # (B, num_classes)
        return x

In [15]:
model=AlexNet(3,3)

In [16]:
number_of_epochs=10
learning_rate=0.001

#loss function
criterion=nn.CrossEntropyLoss()

#optimizer
optimizer=optim.Adam(model.parameters(),learning_rate)

In [19]:
total_step = len(train_dataloader)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

for epoch in range(number_of_epochs):
    model.train()
    for images, labels in tqdm(train_dataloader):  
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images,labels in validation_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()

            # For accuracy calculation (modify if needed for your task)
            _, predicted = torch.max(outputs.data, 1)
            val_total +=labels.size(0)
            val_correct += (predicted ==labels).sum().item()

    # Calculate validation metrics
    avg_val_loss = val_loss / len(validation_dataloader)
    val_accuracy = 100 * val_correct / val_total

    print(f'Epoch [{epoch+1}/{number_of_epochs}], '
          f'Train Loss: {loss.item():.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, '
          f'Val Acc: {val_accuracy:.2f}%')

print('Finished training')

100%|█████████████████████████████████████████| 223/223 [05:32<00:00,  1.49s/it]


Epoch [1/10], Train Loss: 1.5391, Val Loss: 1.0645, Val Acc: 41.24%


100%|█████████████████████████████████████████| 223/223 [05:47<00:00,  1.56s/it]


Epoch [2/10], Train Loss: 1.6237, Val Loss: 1.0651, Val Acc: 41.24%


100%|█████████████████████████████████████████| 223/223 [05:51<00:00,  1.57s/it]


...

Epoch [8/10], Train Loss: 1.5837, Val Loss: 1.0644, Val Acc: 41.24%


100%|█████████████████████████████████████████| 223/223 [05:50<00:00,  1.57s/it]


Epoch [9/10], Train Loss: 1.5723, Val Loss: 1.0643, Val Acc: 41.24%


100%|█████████████████████████████████████████| 223/223 [05:54<00:00,  1.59s/it]


Epoch [10/10], Train Loss: 1.5780, Val Loss: 1.0642, Val Acc: 41.24%
Finished training


In [20]:
torch.save(model.state_dict(), './alexNet_MP.pth')